## 📌 Understanding `RunnableSequence` in LangChain

### **What is `RunnableSequence`?**
✅ `RunnableSequence` is a **workflow builder** that **chains multiple steps together** in a sequence.  
✅ It allows **LLMs, embeddings, retrievers, functions, and tools** to work **sequentially** in LangChain.  
✅ Works as a **replacement for `SequentialChain`**, which is now deprecated.

---

## **🔹 Why Use `RunnableSequence`?**
✔ **Standardized execution** → Works with LLMs, retrievers, and Python functions.  
✔ **Preserves data flow** → Each step's output becomes the next step’s input.  
✔ **More flexibility than `SequentialChain`** → Supports **parallel processing, async execution, and streaming**.

- https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html

---

In [ ]:
# read api_key from file
with open('../api_keys.txt', 'r') as file:
    api_key = file.read()


# Set loaded api_key as OPENAI_API_KEY environmental variable
import os
os.environ["OPENAI_API_KEY"] = api_key

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages( ("system", "Summarize the following {text} in 2 bullet points") )


llm = ChatOpenAI(model='gpt-3.5-turbo')

In [9]:
from langchain.schema.runnable import RunnableSequence

# Create a sequential workflow
seq_chain = RunnableSequence(
    prompt,
    llm
)


output = seq_chain.invoke({"text": "The resulting RunnableSequence is itself a runnable, which means it can be invoked, streamed, or further chained just like any other runnable. Advantages of chaining runnables in this way are efficient streaming (the sequence will stream output as soon as it is available), and debugging and tracing with tools like LangSmith."})
print(output.content)

- The resulting RunnableSequence is a runnable that can be invoked, streamed, or further chained like any other runnable.
- Chaining runnables in this way allows for efficient streaming of output and enables debugging and tracing with tools like LangSmith.


### ☕️ Or simpler Syntax

In [ ]:
chain = prompt | llm


output = chain.invoke({"text": "The resulting RunnableSequence is itself a runnable, which means it can be invoked, streamed, or further chained just like any other runnable. Advantages of chaining runnables in this way are efficient streaming (the sequence will stream output as soon as it is available), and debugging and tracing with tools like LangSmith."})
print(output.content)

- The RunnableSequence is a runnable that can be invoked, streamed, or further chained like any other runnable.
- Chaining runnables in this way allows for efficient streaming of output and enables debugging and tracing with tools like LangSmith.


## **What is `RunnableLambda`?**
`RunnableLambda` is a **lightweight wrapper** that allows **any Python function** to be used as a **step in a LangChain pipeline**. It helps in **data transformations, preprocessing, and custom logic** between AI model interactions.

---

## **🔹 Why Use `RunnableLambda`?**
✅ **Makes Python functions compatible** with LangChain's `Runnable` framework.  
✅ **Can be used in AI workflows** for **text transformation, data filtering, logging, and conditional branching**.  
✅ **Works in sequence with LLMs, retrievers, or databases**.

- https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableLambda.html

---

### **👀 Example 1: Uppercase Runnable**
🔹 **Scenario:** Convert input to uppercase

In [ ]:
from langchain.schema.runnable import RunnableLambda


# Define a simple function
def to_uppercase(text: str) -> str:
    return text.upper()


# Wrap it in a RunnableLambda
uppercase_runnable = RunnableLambda(to_uppercase) 



# Execute it
output = uppercase_runnable.invoke("hello langchain")
print(output) 

HELLO LANGCHAIN


### **👀 Example 2: Text Processing Pipeline**
🔹 **Scenario:** Convert input to uppercase → Reverse text → Count words.

In [21]:
from langchain.schema.runnable import RunnableLambda

# Step 1: Convert text to uppercase
def to_uppercase(data: dict) -> dict:
    return {"text": data["text"].upper()}



# Step 2: Reverse the text
def reverse_text(data: dict) -> dict:
    return {"text": data["text"][::-1]}



# Step 3: Count words
def count_words(data: dict) -> dict:
    return {"word_count": len(data["text"].split())}

### 🖨️ Printing all middle stages

In [22]:
def print_runnable(data: dict) -> dict:
    print(data)
    return data

In [23]:
# Create a sequential workflow
chain = RunnableSequence(
    RunnableLambda(to_uppercase) ,
    RunnableLambda(print_runnable) ,
    RunnableLambda(reverse_text) ,
    RunnableLambda(print_runnable) ,
    RunnableLambda(count_words)
)


output = chain.invoke({"text": "Hello dear LangChain!"})
print(output) 

{'text': 'HELLO DEAR LANGCHAIN!'}
{'text': '!NIAHCGNAL RAED OLLEH'}
{'word_count': 3}


### ➿ Using `|` operator

In [ ]:
chain = RunnableLambda(to_uppercase) | RunnableLambda(reverse_text) | RunnableLambda(count_words)
output = chain.invoke({"text": "Hello dear LangChain!"})
print(output)  

{'word_count': 3}


### ➿ Using `Lambda fucntion` on multiple lines
- Don't forget to put all the steps within `paranthesis`

In [34]:
chain = (RunnableLambda(to_uppercase) 
        | RunnableLambda(reverse_text) 
        | RunnableLambda(count_words)
        )


output = chain.invoke({"text": "Hello dear LangChain!"})
print(output)  

{'word_count': 3}


### ➿ Doing the same but in one-liner `lambda` functions
- You can return of each `lambda` fucntion as a `dictionary` or a `string`

In [11]:
runnbale_upper_case = RunnableLambda(lambda x: {'text': x['text'].upper()} ) 
runnbale_reverse = RunnableLambda(lambda x: {'text': x['text'][::-1]} ) 
runnabel_count_words = RunnableLambda(lambda x: {'count': len(x['text'].split())} )


chain = runnbale_upper_case  | print_runnable | runnbale_reverse | print_runnable |runnabel_count_words



output = chain.invoke({"text": "Hello LangChain!"})
print(output)  # Output: {'word_count': 2}

{'text': 'HELLO LANGCHAIN!'}
{'text': '!NIAHCGNAL OLLEH'}
{'count': 2}


In [12]:
runnbale_upper_case = RunnableLambda(lambda x: x['text'].upper()) 
runnbale_reverse = RunnableLambda(lambda x: x[::-1] ) 
runnabel_count_words = RunnableLambda(lambda x: len(x.split()) )


chain = runnbale_upper_case  | print_runnable | runnbale_reverse | print_runnable |runnabel_count_words



output = chain.invoke({"text": "Hello LangChain!"})
print(output)  # Output: {'word_count': 2}

HELLO LANGCHAIN!
!NIAHCGNAL OLLEH
2


---

## 👀 Example 3: Using `RunnableSequence` with LLMs and **chaining chains**

1. `summary_prompt`: **Summarizes an input document.**  
2. `topics_chain`: **Extracts key topics from the summary.**  
3. `questions_chain`: **Creates follow-up questions for deeper analysis.**  


In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableSequence


# Load LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")

### 🧱 Step 1: Generate Summary Chain

In [43]:
# Step 1: Generate Summary
summary_prompt = ChatPromptTemplate.from_template("Summarize the following text:\n{text}")

summary_prompt.invoke({"text": "LangChain simplifies building AI-powered applications with modular components."})

ChatPromptValue(messages=[HumanMessage(content='Summarize the following text:\nLangChain simplifies building AI-powered applications with modular components.')])

In [44]:
summary_chain = summary_prompt | llm

summary = summary_chain.invoke({"text": "LangChain simplifies building AI-powered applications with modular components."})
summary

AIMessage(content='LangChain streamlines the development of AI applications by providing modular components.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 26, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-868052a9-585d-40fa-97a1-bafc853d87fe-0')

### 🧱 Step 2: Generate Topics chain

In [45]:

topics_prompt = ChatPromptTemplate.from_template("Extract key topics from this summary:\n{summary}")

topics_chain = topics_prompt | llm 

topics = topics_chain.invoke({"summary": summary})
topics

AIMessage(content='- LangChain\n- AI applications\n- Modular components\n- Development\n- Streamlining', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 175, 'total_tokens': 194, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c4b6a78c-6e3a-42a2-a7ea-f4b488807ffd-0')

### 🧱 Step 3: Generate question chain

In [46]:
# Step 3: Generate Follow-Up Questions
questions_prompt = ChatPromptTemplate.from_template("Generate follow-up questions for these topics:\n{topics}")

questions_chain = questions_prompt | llm

questions_chain.invoke({"topics": topics})


AIMessage(content='1. Can you provide more details about LangChain and its specific features?\n2. How are AI applications being integrated with LangChain?\n3. In what ways are modular components being used in development with LangChain?\n4. What are some strategies for streamlining the development process when working with LangChain?\n5. Can you elaborate on the different phases of development involved in implementing LangChain?\n6. How does LangChain facilitate the creation of AI applications?\n7. What are the advantages of using modular components in the context of LangChain?\n8. Are there any specific challenges that developers may face when streamlining the development process with LangChain?\n9. How does LangChain support collaboration and integration of different components in development?\n10. What are some best practices for optimizing the performance of AI applications developed using LangChain?', response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 18

In [ ]:
chain = (
    # Step 1: First Summarization and the pass {"summary": summary from content of previous stage} to the next stage
    summary_chain  # >>> summerize the text, output has content variable
      |  
    # Step 2: Extract topics and pass {"topics": Extract key topics from the pevious summary} to the next stage
    topics_chain # Extract topics from the previous summary, output has content variabl
      |  
    # Step 3: Generate questions
    questions_chain  # Output has content variable
)


# Run the pipeline
output = chain.invoke({"text": "LangChain simplifies building AI-powered applications with modular components."})
print(output.content)

1. Can you explain more about what LangChain is and how it works?
2. What are some specific examples of AI applications that utilize LangChain?
3. How do modular components play a role in the development of LangChain?
4. In what ways does LangChain make the creation process easier compared to traditional methods?


## 🕵️ How to inspect runnables
- https://python.langchain.com/docs/how_to/inspect/

`!pip install grandalf`

In [48]:
chain.get_graph().print_ascii()

    +-------------+    
    | PromptInput |    
    +-------------+    
           *           
           *           
           *           
+--------------------+ 
| ChatPromptTemplate | 
+--------------------+ 
           *           
           *           
           *           
    +------------+     
    | ChatOpenAI |     
    +------------+     
           *           
           *           
           *           
 +------------------+  
 | ChatOpenAIOutput |  
 +------------------+  
           *           
           *           
           *           
+--------------------+ 
| ChatPromptTemplate | 
+--------------------+ 
           *           
           *           
           *           
    +------------+     
    | ChatOpenAI |     
    +------------+     
           *           
           *           
           *           
 +------------------+  
 | ChatOpenAIOutput |  
 +------------------+  
           *           
           *           
           *    

In [49]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='Summarize the following text:\n{text}'))]),
 ChatPromptTemplate(input_variables=['summary'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['summary'], template='Extract key topics from this summary:\n{summary}'))]),
 ChatPromptTemplate(input_variables=['topics'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topics'], template='Generate follow-up questions for these topics:\n{topics}'))])]

---